In [ ]:
!pip install langfuse

In [ ]:
import sys
import os 
import nest_asyncio

# Sanity check
print(sys.executable)
nest_asyncio.apply()

os.environ["LANGFUSE_SECRET_KEY"]="sk-lf-305fb0a1-0d3f-4732-841a-baeed1abc7ff"
os.environ["LANGFUSE_PUBLIC_KEY"]="pk-lf-b044ea0d-bbbb-4e59-9a38-b814356fde31"
os.environ["LANGFUSE_HOST"]="https://cloud.langfuse.com"


from dotenv import load_dotenv
load_dotenv() 


In [ ]:
import os
from pydantic import BaseModel, Field
from llama_index.core.workflow import (
    Workflow,
    step,
    Event,
    Context,
    StartEvent,
    StopEvent
)
from llama_index.llms.openai import OpenAI
from llama_index.core import SimpleDirectoryReader
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    VectorStoreIndex,
    StorageContext,
)
from llama_index.core.node_parser import SentenceSplitter
from llama_index.retrievers.bm25 import BM25Retriever
import Stemmer
from llama_index.core import VectorStoreIndex, get_response_synthesizer
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine
from langfuse.decorators import observe, langfuse_context

In [ ]:
llm = OpenAI(temperature=0.05, model="gpt-4o-mini")


class EchoWorkflow(Workflow):
    """A dummy workflow with only one step sending back the input given."""

    @observe(capture_input=False)
    @step()
    async def run_step(self, ev: StartEvent) -> StopEvent:
        langfuse_context.update_current_trace(
            session_id="0000000",
            metadata={"project": "very_important"}
        )
        message = str(ev.get("message", ""))
        langfuse_context.update_current_observation(
            input=message, 
        )
        documents = SimpleDirectoryReader(
            input_files=["../notebooks/paul_graham_essay.txt"],
        ).load_data()
        splitter = SentenceSplitter(chunk_size=256)
        nodes = splitter.get_nodes_from_documents(documents)
        retriever_top_2 = BM25Retriever.from_defaults(
            nodes=nodes,
            similarity_top_k=2,
            stemmer=Stemmer.Stemmer("english"),
            language="english",
        )
        nodes = retriever_top_2.retrieve(message)
        context = "\n ".join([x.text for x in nodes])
        response = llm.complete(f"Summarize this : {context}")
        return StopEvent(result=response.text)

In [ ]:
w = EchoWorkflow()
r = await w.run(message="computer")

In [ ]:
r

## Get traces

In [ ]:
import langfuse
from langfuse import Langfuse

langfuse = Langfuse()
traces = langfuse.fetch_traces()
traces.data[0]

## Get a dataset

In [ ]:

dataset = langfuse.get_dataset("MyDataset")

In [ ]:
dataset

In [ ]:
for item in dataset.items:
    if str(item.status) == "DatasetStatus.ACTIVE":
        print(item.input, item.expected_output)

In [ ]:
item.__dict__

## A/B testing 

In [ ]:
prompt = langfuse.get_prompt("JokeTemplate1", version=1)
compiled_prompt = prompt.compile(topic="cats")

compiled_prompt

In [ ]:
prompt = langfuse.get_prompt("JokeTemplate1", version=2)
compiled_prompt = prompt.compile(topic="cats")

compiled_prompt